In [1]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [2]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [3]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [4]:
data[0]

'BV1FU411f7AX'

In [5]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1FU411f7AX
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/190453309015469940094d5ad77763724ec3218b62ecfdcaee9d?auth_key=1720585368-ccfeba5022794bfabcdc7438cfacc005-0-48e149ba6e0f4c8d86629ea127119094
Extracted segment: 1Hx4y167B2
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/10036383521516886557971445b1fe1305ef022ff4db56987934?auth_key=1720585373-6349865ef0d24085b19743916a7b0b6d-0-58119b322f7d1e547b0278aa6d8dfa54
Extracted segment: 1Ux4y187nS
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/10033403111512448704033241eea1d30f40e89a16f05f3b79e0?auth_key=1720585377-bd509119df9a4553a77615728467259b-0-782713360e03998436f720c6d61c39af
Extracted segment: 16z4y1w7h2
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/5784052141332564220823b85ecf0

In [19]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)(?:\n\n|$)", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [28]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

template_orig = r"""
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
        参考信息:{ref}
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ### 板书一:     ### 板书二:       ### 讲解一:       ### 讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。### 板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$### 板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$### 讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。### 讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
            d: 对于所有题目尽可能用解释性的语言进行解答，不到万不得已不用公式直接求解，以最大限度上避免超纲的情况发生。
        13.板书不能超过6条，如果6条写不下，使用别的方法来解题。
"""

In [8]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
两个质数的和为39，这两个质数分别是多少？

### 讲解：
我们假设这两个质数分别是A和B，且A + B = 39。首先，我们需要注意到39是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是37。因此，这两个质数分别是37和2。

### 相似题一：
两个质数的和为41，这两个质数分别是多少？

### 相似题一讲解：
我们假设这两个质数分别是A和B，且A + B = 41。首先，我们需要注意到41是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是41 - 2 = 39。但是39不是质数，所以我们需要重新考虑。我们可以尝试其他组合，最终发现41可以分解为两个质数的和，即5和36，但36不是质数。继续尝试，最终发现41可以分解为两个质数的和，即11和30，但30也不是质数。最后，我们发现41可以分解为两个质数的和，即3和38，但38也不是质数。经过多次尝试，我们发现41不能分解为两个质数的和。因此，这个题目没有符合条件的答案。executing
### 原题：
有五元、十元的人民币共14张，共100元。五元和十元各几张？

### 讲解：
这里总共有14张人民币。我们假设这14张人民币全是五元的，那么总金额就是14×5=70元。但实际总金额是100元，所以这里有一个差额。这个差额是因为我们把十元的人民币当成了五元的人民币。每张十元的人民币比五元的多5元，所以总金额会少掉5元。总共少掉的金额是100-70=30元。我们可以通过这个差额来计算十元的张数。十元的张数就是总共少掉的金额除以每张少掉的金额，即30÷5=6张。所以十元的有6张。五元的张数就是总数14减去十元的张数6，等于8张。

### 相似题一：
有两元、五元的人民币共20张，共60元。两元和五元各几张？

### 相似题一讲解：
这里总共有20张人民币。我们假设这20张人民币全是两元的，那么总金额就是20×2=40元。但实际总金额是60元，所以这里有一个差额。这个差

In [16]:
raw_problem_answer

['### 原题：\n两个质数的和为39，这两个质数分别是多少？\n\n### 讲解：\n我们假设这两个质数分别是A和B，且A + B = 39。首先，我们需要注意到39是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是37。因此，这两个质数分别是37和2。\n\n### 相似题一：\n两个质数的和为41，这两个质数分别是多少？\n\n### 相似题一讲解：\n我们假设这两个质数分别是A和B，且A + B = 41。首先，我们需要注意到41是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是41 - 2 = 39。但是39不是质数，所以我们需要重新考虑。我们可以尝试其他组合，最终发现41可以分解为两个质数的和，即5和36，但36不是质数。继续尝试，最终发现41可以分解为两个质数的和，即11和30，但30也不是质数。最后，我们发现41可以分解为两个质数的和，即3和38，但38也不是质数。经过多次尝试，我们发现41不能分解为两个质数的和。因此，这个题目没有符合条件的答案。',
 '### 原题：\n有五元、十元的人民币共14张，共100元。五元和十元各几张？\n\n### 讲解：\n这里总共有14张人民币。我们假设这14张人民币全是五元的，那么总金额就是14×5=70元。但实际总金额是100元，所以这里有一个差额。这个差额是因为我们把十元的人民币当成了五元的人民币。每张十元的人民币比五元的多5元，所以总金额会少掉5元。总共少掉的金额是100-70=30元。我们可以通过这个差额来计算十元的张数。十元的张数就是总共少掉的金额除以每张少掉的金额，即30÷5=6张。所以十元的有6张。五元的张数就是总数14减去十元的张数6，等于8张。\n\n### 相似题一：\n有两元、五元的人民币共20张，共60元。两元和五元各几张？\n\n### 相似题一讲解：\n这里总共有20张人民币。我们假设这20张人民币全是两元的，那么总金额就是20×2=40元。但实际总金额是60元，所以这里有一

In [20]:
extract_question_explanations(raw_problem_answer)

In [21]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '两个质数的和为39，这两个质数分别是多少？',
  'original_explanation': '我们假设这两个质数分别是A和B，且A + B = 39。首先，我们需要注意到39是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是37。因此，这两个质数分别是37和2。',
  'similar_question_1': '两个质数的和为41，这两个质数分别是多少？',
  'similar_explanation_1': '我们假设这两个质数分别是A和B，且A + B = 41。首先，我们需要注意到41是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是41 - 2 = 39。但是39不是质数，所以我们需要重新考虑。我们可以尝试其他组合，最终发现41可以分解为两个质数的和，即5和36，但36不是质数。继续尝试，最终发现41可以分解为两个质数的和，即11和30，但30也不是质数。最后，我们发现41可以分解为两个质数的和，即3和38，但38也不是质数。经过多次尝试，我们发现41不能分解为两个质数的和。因此，这个题目没有符合条件的答案。'},
 {'original_question': '有五元、十元的人民币共14张，共100元。五元和十元各几张？',
  'original_explanation': '这里总共有14张人民币。我们假设这14张人民币全是五元的，那么总金额就是14×5=70元。但实际总金额是100元，所以这里有一个差额。这个差额是因为我们把十元的人民币当成了五元的人民币。每张十元的人民币比五元的多5元，所以总金额会少掉5元。总共少掉的金额是100-70=30元。我们可以通过这个差额来计算十元的张数。十元的张数就是总共少掉的金额除以每张少掉的金额，即30÷5=6张。所以十元的有6张。五元的张数就是总数14减去十元的张数6，等于8张。',
  'similar_question_1': '有两元、五元的

In [22]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n两个质数的和为39，这两个质数分别是多少？\n\n讲解:\n我们假设这两个质数分别是A和B，且A + B = 39。首先，我们需要注意到39是一个奇数。根据数的性质，只有奇数加偶数才能等于奇数。因此，这两个质数中必须有一个是奇数，另一个是偶数。再满足第二个条件，即两个数都要是质数。我们知道唯一的偶质数是2。那么，A或B中有一个是2，另一个数就是37。因此，这两个质数分别是37和2。\n\n题目:\n有五元、十元的人民币共14张，共100元。五元和十元各几张？\n\n讲解:\n这里总共有14张人民币。我们假设这14张人民币全是五元的，那么总金额就是14×5=70元。但实际总金额是100元，所以这里有一个差额。这个差额是因为我们把十元的人民币当成了五元的人民币。每张十元的人民币比五元的多5元，所以总金额会少掉5元。总共少掉的金额是100-70=30元。我们可以通过这个差额来计算十元的张数。十元的张数就是总共少掉的金额除以每张少掉的金额，即30÷5=6张。所以十元的有6张。五元的张数就是总数14减去十元的张数6，等于8张。\n\n题目:\n一个数的小数点向右移动一位比原数就大135，求原数是多少？\n\n讲解:\n根据题意，小数点向右移动一位，数值会变成原数的十倍。我们可以举例子来看，比如2.5，小数点向右移动一位变成25，0.5向右移动一位变成5，前后对比，变化后的数字是原数的十倍。所有这样的数字都是这样，只要小数点向右移动一位，变化后的数是原数的十倍。\n\n根据题目，我们可以得到变化后的数是原数的十倍。变化后的数比原数多135，这就是差倍问题。我们假设原数是x，那么变化后的数就是10x。根据题意，10x比x多135，所以我们可以列出方程：\n\n10x - x = 135\n\n解这个方程：\n\n9x = 135\n\nx = 135 / 9\n\nx = 15\n\n所以，原数是15。我们可以验算一下，15小数点向右移动一位变成150，150比15大135，验证正确。\n\n'

In [23]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [29]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig, max_tokens=2000))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书[二三四五六七八九十]+:[\s\S]*?)*?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl, max_tokens=2000))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是质数的定义和质数的性质，解题关键点是找到两个质数的和为41。

### 板书一:
设这两个质数分别为$p$和$q$，
依题意得：
$p + q = 41$

### 板书二:
$p$和$q$都是质数，
且$p \leq q$，
则$p \leq 20.5$，
即$p \leq 20$

### 板书三:
从小到大依次检查小于等于20的质数：
2, 3, 5, 7, 11, 13, 17, 19

### 板书四:
逐一检查这些质数是否与另一个质数的和为41：
$p = 2, q = 41 - 2 = 39$ (39不是质数)
$p = 3, q = 41 - 3 = 38$ (38不是质数)
$p = 5, q = 41 - 5 = 36$ (36不是质数)
$p = 7, q = 41 - 7 = 34$ (34不是质数)
$p = 11, q = 41 - 11 = 30$ (30不是质数)
$p = 13, q = 41 - 13 = 28$ (28不是质数)
$p = 17, q = 41 - 17 = 24$ (24不是质数)
$p = 19, q = 41 - 19 = 22$ (22不是质数)

### 板书五:
$p = 19, q = 41 - 19 = 22$ (22不是质数)
$p = 23, q = 41 - 23 = 18$ (18不是质数)
$p = 29, q = 41 - 29 = 12$ (12不是质数)
$p = 31, q = 41 - 31 = 10$ (10不是质数)
$p = 37, q = 41 - 37 = 4$ (4不是质数)
$p = 41, q = 41 - 41 = 0$ (0不是质数)

### 板书六:
$p = 2, q = 41 - 2 = 39$ (39不是质数)
$p = 3, q = 41 - 3 = 38$ (38不是质数)
$p = 5, q = 41 - 5 = 36$ (36不是质数)
$p = 7, q = 41 - 7 = 34$ (34不是质数)
$p = 11, q = 41 - 11 = 30$ (30不是质数)
$p = 13, q = 41 - 13 = 28$ (28不是质数)
$p = 17, q = 41 - 17 = 

In [30]:
with open("compare_results_p5.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [15]:
print(len(orig_expls))

1


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

